# Visualizing MAAP STAC Dataset with MosaicJSON

Authors: Samuel Ayers (UAH), Alex Mandel (DevSeed), Aimee Barciauskas (DevSeed)

Date: July 23, 2021

Description: In this notebook, we visualize SRTM Cloud-Optimized GeoTIFFs (COGs) from MAAP's STAC using a generated mosaic from MAAP's TiTiler.

## Run This Notebook
To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the ["Getting started with the MAAP"](https://docs.maap-project.org/en/latest/getting_started/getting_started.html) section of our documentation.

Disclaimer: it is highly recommended to run a tutorial within MAAP's ADE, which already includes packages specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors.

## Note About the Data

The NASA Shuttle Radar Topographic Mission (SRTM) has provided digital elevation data (DEMs) for over 80% of the globe. This data is currently distributed free of charge by USGS and is available for download from the National Map Seamless Data Distribution System, or the USGS FTP site.

At MAAP, we've converted this elevation data into Cloud-Optimized GeoTIFFs (COGs) so they can be efficiently queried and visualized. These COGs are available in the [MAAP STAC](https://stac.maap-project.org/collections/SRTMGL1_COD/items).

## Additional Resources
- [USGS EROS Archive - Shuttle Radar Topography Mission (SRTM)](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-digital-elevation-shuttle-radar-topography-mission-srtm)
- [TiTiler API Documentation](https://astrogeology.usgs.gov/titiler/docs)
- [Github - MosaicJSON](https://github.com/developmentseed/mosaicjson-spec)
- [Working with MosaicJSON - TiTiler (DevSeed Example)](https://developmentseed.org/titiler/examples/notebooks/Working_with_MosaicJSON/)

## Importing and Installing Packages

To be able to run this notebook you'll need the following requirements:

- rasterio
- folium
- requests
- tqdm
- rio-tiler (2.0b8) (Optional)
- cogeo-mosaic (Optional)

If the packages below are not installed already, uncomment the following cell:

In [ ]:
# !pip install rasterio folium requests tqdm
# !pip install rio-tiler cogeo-mosaic --pre

In [ ]:
import requests
from pprint import pprint

from rio_tiler.io import COGReader
from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

## Fetch SRTM COG STAC items

In [2]:
stac_endpoint = "https://stac.maap-project.org/search"

stac_json = {
    "collections": ["SRTMGL1_COD"],
    "bbox": [4,42,16,48],
    "limit": 120
}

headers = {
            "Content-Type": "application/json",
            "Accept": "application/geo+json, application/json",
        }

# Read Items

r_stac = requests.post(stac_endpoint, headers=headers, json=stac_json)
items = r_stac.json()

Map the data bounds:

In [3]:
geojson = {'type': 'FeatureCollection', 'features': items['features']}

bounds = featureBounds(geojson)
zoom_start = 5

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=zoom_start
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

## Create Mosaic

We're using the TiTiler deployed by MAAP

In [4]:
titiler_endpoint = "https://titiler.maap-project.org"  # MAAP titiler endpoint

To begin, we'll pull our COG:

In [5]:
%time
from rio_tiler.io import COGReader

first_cog = geojson['features'][0]['assets']['cog_default']['href']
with COGReader(first_cog) as cog:
    info = cog.info()

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.87 µs


Next, we will create the mosaic using the geojson from above. SRTMGL1 COGs have a "cog default" asset type, so we can create a mosaic of these type="image/tiff" assets. We can get access to these assets by using the accessor function.

In [6]:
from cogeo_mosaic.mosaic import MosaicJSON

mosaicdata = MosaicJSON.from_features(geojson.get('features'), minzoom=6, maxzoom=info.maxzoom, accessor=lambda feature : feature['assets']['cog_default']['href'])

Now we will upload the mosaicjson to the TiTiler:

In [7]:
r = requests.post(
    url=f"{titiler_endpoint}/mosaics",
    headers={
        "Content-Type": "application/vnd.titiler.mosaicjson+json",
    },
    json=mosaicdata.dict(exclude_none=True)).json()

pprint(r)

{'id': '90b3f48f-d34d-4ee0-9e32-13c27c6325a3',
 'links': [{'href': 'https://titiler.maap-project.org/mosaics/90b3f48f-d34d-4ee0-9e32-13c27c6325a3',
            'rel': 'self',
            'title': 'Self',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.org/mosaics/90b3f48f-d34d-4ee0-9e32-13c27c6325a3/mosaicjson',
            'rel': 'mosaicjson',
            'title': 'MosaicJSON',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.org/mosaics/90b3f48f-d34d-4ee0-9e32-13c27c6325a3/tilejson.json',
            'rel': 'tilejson',
            'title': 'TileJSON',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.org/mosaics/90b3f48f-d34d-4ee0-9e32-13c27c6325a3/tiles/{z}/{x}/{y}',
            'rel': 'tiles',
            'title': 'Tiles',
            'type': 'application/json'},
           {'href': 'https://titiler.maap-project.org/mosaics/90b3f48f-d34d-4ee0-9e32-13c27c6325

The response from the post request gives endpoints for different services (eg. mosaicjson, tilejson, tiles, wmts, etc). We're fetching the `tilejson` endpoint. 

In [8]:
tilejson_endpoint = list(filter(lambda x: x.get("rel") == "tilejson", dict(r)["links"]))

## Display Tiles

NOTE: You have to zoom to "minzoom" to load the data.

SECOND NOTE: The important bit is the "tiles" endpoint returned from `f"{titiler_endpoint}/mosaics`. This endpoint (e.g. `https://titiler.maap-project.org/mosaics/4199126b-9313-435a-b4b5-17802716b7b1/tiles/{z}/{x}/{y}`) could be used in any map client which can tile `x,y,z` layers.

In [9]:
r_te = requests.get(
    tilejson_endpoint[0].get('href')
).json()

pprint(r_te)

tiles = TileLayer(
    tiles=f"{r_te['tiles'][0]}?rescale=0,4000",
    min_zoom=r_te["minzoom"],
    max_zoom=r_te["maxzoom"],
    opacity=1,
    attr="USGS"
)

tiles.add_to(m)
m

{'bounds': [2.999861111111111,
            40.999861111111116,
            17.000138888888888,
            49.00013888888889],
 'center': [10.0, 45.0, 6],
 'maxzoom': 12,
 'minzoom': 6,
 'name': '90b3f48f-d34d-4ee0-9e32-13c27c6325a3',
 'scheme': 'xyz',
 'tilejson': '2.2.0',
 'tiles': ['https://titiler.maap-project.org/mosaics/90b3f48f-d34d-4ee0-9e32-13c27c6325a3/tiles/{z}/{x}/{y}@1x'],
 'version': '1.0.0'}
